In [3]:
import glob
import numpy as np
import xarray as xr
import pandas as pd

# User inputs
path = '/g/data/hh5/WS2019/ccc561/ctl01/'
var='t'
file_template = f's{var}[0-9][0-9]_ctl01.nc'

files=sorted(glob.glob(path+file_template))
files

pressure = xr.DataArray(np.asarray([1000,925,850,700,600,500,400,300,250,200,150,100,70,50,30,20,10,5],dtype=np.int32),dims=('lev',),attrs={'units':'hPa'})
height = xr.DataArray(np.asarray([165,300,575,990,1535,2215,3025,3970,5070,6335,7780,9440,11360,13650,16550,20600,27360,36355],dtype=np.int32), dims=('lev',),attrs={'units':'m'})
pressure

<xarray.DataArray (lev: 18)>
array([1000,  925,  850,  700,  600,  500,  400,  300,  250,  200,  150,  100,
         70,   50,   30,   20,   10,    5], dtype=int32)
Dimensions without coordinates: lev
Attributes:
    units:    hPa

In [22]:
ds= xr.open_mfdataset(files,)
ds = ds.where(ds < 9.e36)
if var == 't':
    var = 'temp'

In [ ]:
for dd in ds.data_vars:
    print(dd)

In [23]:
t=xr.concat([ds[dd] for dd in ds.data_vars],dim='lev')
t.coords['lev']=pressure
t

<xarray.DataArray 't01' (lev: 18, year: 50, month: 12, latitude: 56, longitude: 64)>
dask.array<shape=(18, 50, 12, 56, 64), dtype=float32, chunksize=(1, 50, 12, 56, 64)>
Coordinates:
  * latitude   (latitude) float32 -87.56133 -84.402245 ... 84.402245 87.56133
  * longitude  (longitude) float32 0.0 5.625 11.25 ... 343.125 348.75 354.375
  * month      (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * year       (year) int32 1 2 3 4 5 6 7 8 9 10 ... 42 43 44 45 46 47 48 49 50
  * lev        (lev) int64 1000 925 850 700 600 500 400 ... 100 70 50 30 20 10 5
Attributes:
    units:    K

In [24]:
# Annual mean
days=xr.DataArray([31,28,31,30,31,30,31,31,30,31,30,31],dims=('month',),coords={'month':t['month']})/365
clim = t *days
clim[0,0,0,0,0].values

array(nan)

In [25]:
clim1 = clim.sum(dim='month')
clim1[0,0,0,0].values

array(0.)

In [6]:
clim[0,0,:,0,0].values

array([8.46700025e+35, 7.64761313e+35, 8.46700025e+35, 8.19387121e+35,
       8.46700025e+35, 8.19387121e+35, 8.46700025e+35, 8.46700025e+35,
       8.19387121e+35, 8.46700025e+35, 8.19387121e+35, 8.46700025e+35])

In [38]:
clim1 = clim1.where(t < 9.e36)
clim1 = clim1.sel(month=1).drop('month')
clim1

<xarray.DataArray (lev: 18, year: 50, latitude: 56, longitude: 64)>
dask.array<shape=(18, 50, 56, 64), dtype=float64, chunksize=(1, 50, 56, 64)>
Coordinates:
  * latitude   (latitude) float32 -87.56133 -84.402245 ... 84.402245 87.56133
  * longitude  (longitude) float32 0.0 5.625 11.25 ... 343.125 348.75 354.375
  * year       (year) int32 1 2 3 4 5 6 7 8 9 10 ... 42 43 44 45 46 47 48 49 50
  * lev        (lev) int64 1000 925 850 700 600 500 400 ... 100 70 50 30 20 10 5

In [39]:
clim1 = clim1.rename('temp')

In [40]:
clim1.assign_attrs({'long_name': f'Annual-mean atmosphere {var}','units':t.units})

<xarray.DataArray 'temp' (lev: 18, year: 50, latitude: 56, longitude: 64)>
dask.array<shape=(18, 50, 56, 64), dtype=float64, chunksize=(1, 50, 56, 64)>
Coordinates:
  * latitude   (latitude) float32 -87.56133 -84.402245 ... 84.402245 87.56133
  * longitude  (longitude) float32 0.0 5.625 11.25 ... 343.125 348.75 354.375
  * year       (year) int32 1 2 3 4 5 6 7 8 9 10 ... 42 43 44 45 46 47 48 49 50
  * lev        (lev) int64 1000 925 850 700 600 500 400 ... 100 70 50 30 20 10 5
Attributes:
    long_name:  Annual-mean atmosphere temp
    units:      K

In [41]:
clim1.to_netcdf('/Users/Claire/mount/raij_home/gdata5/WS2019/ccc561/ctl01/test_st_concat.nc')